In [ ]:
import os
from keras import layers, optimizers, models
from keras.applications.resnet import ResNet152, preprocess_input
from keras.layers import *    
from keras.models import Model

In [ ]:
train_dir = os.path.join(r'D:\jupyterlab\training')
validation_dir = os.path.join(r'D:\jupyterlab\val')

In [ ]:
from keras.callbacks import ReduceLROnPlateau,EarlyStopping
Reduce=ReduceLROnPlateau(
    monitor ='val_loss',#监测的值，可以是accuracy，val_loss,val_accuracy
    factor=0.1,#缩放学习率的值，学习率将以lr = lr*factor的形式被减少
    patience=2,#当patience个epoch过去而模型性能不提升时，学习率减少的动作会被触发
    verbose=1,
    mode='auto',#‘auto’，‘min’，‘max’之一 默认‘auto’就行
    cooldown=0,#学习率减少后，会经过cooldown个epoch才重新进行正常操作
    min_lr=0 #学习率最小值，能缩小到的下限
)
early_stop = EarlyStopping(
    monitor='val_loss', 
    patience=10
)

In [ ]:
resnet152 = ResNet152(weights='imagenet', include_top=False, input_shape=(300,300, 3))

model = models.Sequential()
model.add(resnet152)
model.add(GlobalAveragePooling2D())
model.add(Dense(1024))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))
# model.add(layers.Flatten())
# model.add(Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

resnet152.trainable = False
#冻结一个层意味着将其排除在训练之外，即其权重将永远不会更新
optimizer = optimizers.RMSprop(lr=1e-4)

def get_lr_metric(optimizer):
    def lr(y_true, y_pred):
        return optimizer.lr
    return lr
lr_metric = get_lr_metric(optimizer)
model.compile(loss='binary_crossentropy', optimizer = optimizers.RMSprop(lr=1e-4), metrics=['acc',lr_metric])

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet152 (Functional)       (None, 10, 10, 2048)      58370944  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              2098176   
_________________________________________________________________
batch_normalization (BatchNo (None, 1024)              4096      
_________________________________________________________________
activation (Activation)      (None, 1024)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               2

In [6]:
from keras.preprocessing.image import ImageDataGenerator

batch_size = 64

train_datagen = ImageDataGenerator(
    rotation_range=45,#随机旋转
    width_shift_range=0.2,#是图像在水平上平移的范围
    height_shift_range=0.2,#垂直方向上平移的范围
    shear_range=0.2,#随机错切变换的角度
    zoom_range=0.2,#随机缩放的范围
    horizontal_flip=True,#随机将图像水平翻转
    preprocessing_function=preprocess_input#归一化
)
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)


train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(300,300),
        batch_size=batch_size,
        class_mode='binary')
validation_generator = val_datagen.flow_from_directory(
        validation_dir,
        target_size=(300,300),
        batch_size=batch_size,
        class_mode='binary')

Found 8005 images belonging to 2 classes.
Found 4000 images belonging to 2 classes.


In [7]:
history = model.fit(
      train_generator,
      steps_per_epoch=train_generator.samples//batch_size,
      epochs=30,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples//batch_size,
      callbacks=[early_stop,Reduce]
)

Epoch 1/30
125/125 [==============================] - 2674s 21s/step - loss: 0.0985 - acc: 0.9659 - lr: 1.0000e-04 - val_loss: 0.0257 - val_acc: 0.9945 - val_lr: 1.0000e-04
Epoch 2/30
125/125 [==============================] - 2571s 21s/step - loss: 0.0484 - acc: 0.9825 - lr: 1.0000e-04 - val_loss: 0.0143 - val_acc: 0.9962 - val_lr: 1.0000e-04
Epoch 3/30
125/125 [==============================] - 2575s 21s/step - loss: 0.0371 - acc: 0.9875 - lr: 1.0000e-04 - val_loss: 0.0112 - val_acc: 0.9972 - val_lr: 1.0000e-04
Epoch 4/30
  3/125 [..............................] - ETA: 18:54 - loss: 0.0679 - acc: 0.9688 - lr: 1.0000e-04

KeyboardInterrupt: 

In [8]:
model.save('cat-dog-resnet152-epoch3.h5')

In [19]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet152 (Functional)       (None, 10, 10, 2048)      58370944  
_________________________________________________________________
global_average_pooling2d_3 ( (None, 2048)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
batch_normalization_6 (Batch (None, 1024)              4096      
_________________________________________________________________
activation_6 (Activation)    (None, 1024)              0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_10 (Dense)             (None, 256)              

In [10]:
import os
import numpy as np
from keras import layers, optimizers, models
from keras.applications.resnet import ResNet152, preprocess_input
from keras.layers import *    
from keras.models import Model
from keras.models import load_model
optimizer = optimizers.RMSprop(lr=1e-4)
def get_lr_metric(optimizer):
    def lr(y_true, y_pred):
        return optimizer.lr
    return lr
lr_metric = get_lr_metric(optimizer)
testmodel = load_model('cat-dog-resnet152-epoch12.h5',custom_objects={'lr': lr_metric})

In [11]:
train_dir = os.path.join(r'D:\jupyterlab\training')
validation_dir = os.path.join(r'D:\jupyterlab\val')

In [12]:
from keras.preprocessing.image import ImageDataGenerator

batch_size = 64

train_datagen = ImageDataGenerator(
    rotation_range=45,#随机旋转
    width_shift_range=0.2,#是图像在水平上平移的范围
    height_shift_range=0.2,#垂直方向上平移的范围
    shear_range=0.2,#随机错切变换的角度
    zoom_range=0.2,#随机缩放的范围
    horizontal_flip=True,#随机将图像水平翻转
    preprocessing_function=preprocess_input#归一化
)
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)


train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(300,300),
        batch_size=batch_size,
        class_mode='binary')
validation_generator = val_datagen.flow_from_directory(
        validation_dir,
        target_size=(300,300),
        batch_size=batch_size,
        class_mode='binary')

Found 8005 images belonging to 2 classes.
Found 4000 images belonging to 2 classes.


In [13]:
from keras.callbacks import ReduceLROnPlateau,EarlyStopping
Reduce=ReduceLROnPlateau(
    monitor ='val_loss',#监测的值，可以是accuracy，val_loss,val_accuracy
    factor=0.1,#缩放学习率的值，学习率将以lr = lr*factor的形式被减少
    patience=2,#当patience个epoch过去而模型性能不提升时，学习率减少的动作会被触发
    verbose=1,
    mode='auto',#‘auto’，‘min’，‘max’之一 默认‘auto’就行
    cooldown=0,#学习率减少后，会经过cooldown个epoch才重新进行正常操作
    min_lr=0 #学习率最小值，能缩小到的下限
)
early_stop = EarlyStopping(
    monitor='val_loss', 
    patience=10
)

In [ ]:
history = testmodel.fit(
      train_generator,
      steps_per_epoch=train_generator.samples//batch_size,
      epochs=30,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples//batch_size,
      callbacks=[early_stop,Reduce]
)

Epoch 1/30
125/125 [==============================] - 2609s 21s/step - loss: 0.0164 - acc: 0.9937 - lr: 1.0000e-04 - val_loss: 0.0040 - val_acc: 0.9987 - val_lr: 1.0000e-04
Epoch 2/30
125/125 [==============================] - 2571s 21s/step - loss: 0.0138 - acc: 0.9961 - lr: 1.0000e-04 - val_loss: 0.0028 - val_acc: 0.9992 - val_lr: 1.0000e-04
Epoch 3/30
125/125 [==============================] - 2569s 21s/step - loss: 0.0168 - acc: 0.9941 - lr: 1.0000e-04 - val_loss: 0.0033 - val_acc: 0.9990 - val_lr: 1.0000e-04
Epoch 4/30
125/125 [==============================] - ETA: 0s - loss: 0.0134 - acc: 0.9956 - lr: 1.0000e-04 
Epoch 00004: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
125/125 [==============================] - 2570s 21s/step - loss: 0.0134 - acc: 0.9956 - lr: 1.0000e-04 - val_loss: 0.0031 - val_acc: 0.9992 - val_lr: 1.0000e-04
Epoch 5/30
125/125 [==============================] - 2885s 23s/step - loss: 0.0152 - acc: 0.9943 - lr: 1.0000e-04 - val_loss: 0.0

In [9]:
testmodel.save('cat-dog-resnet152-epoch20.h5')